In [1]:
# import saved modules
from modules.KorEDA.eda import EDA
from modules.preprocess_for_kobert import preprocess

In [2]:
import pandas as pd
import numpy as np
import re

from modules.preprocess_for_kobert import preprocess

In [3]:
# pandas 설정: 각 column 별 (혹은 한 cell에서) 출력되는 글자수 제한을 없애기
pd.set_option('display.max_colwidth', -1)
pd.__version__

'1.1.4'

### Bring Dataset

In [4]:
DATA_PATH = "./data"

In [5]:
df_original = pd.read_csv(f"{DATA_PATH}/몽데이크_Open.csv", encoding="utf-8")
df_original.sample(2)

,qplay_question_id,text,qtid,name,subtitle,question_difficulty,question_type_difficulty,grade,purpose
4700,9422,"원종고등학교에서 학생들이 등교할 때 이용하는 교통수단을 조사하였더니 대중교통을 이용하는 학생들이 전체의 `60%`이고, 대중교통을 이용하는 남자학생은 전체의 `20%`이었다. 이 학교의 학생 중에서 임의로 뽑은 한 명이 대중교통을 이용할 때, 그 학생이 남자 학생일 확률은?",HSTA-04-02,조건부확률,NaN,3,2.0,10,open
5019,25195,"문자카드 `H`, `O`, `N`, `G`, `I`, `K` `6`장을 일렬로 나열할 때, 모음이 이웃하는 경우의 수는 `m`가지, 모음이 양 끝에 오는 경우의 수는 `n`가지이다. `m+n`의 값은?",H1S2-07-11,자리에 대한 조건이 있는 순열의 수,NaN,3,NaN,10,open


In [6]:
df_train = pd.read_csv(f"{DATA_PATH}/data_preprocessed.csv", encoding="utf-8")
df_train.sample(2)

,chapter,qtid,text,math_field,hangul_field,tag_field,name,subtitle,qplay_question_id,question_difficulty,question_type_difficulty,grade,purpose,word2vec_pred_chapter,word2vec_pred_name,tfidf_pred_chapter
3748,HSU1-07,HSU1-07-12,"한 변의 길이가 `4`인 정삼각형 `ABC`에서 세 선분 `AB`, `BC`, `CA`를 모두 `1:3`으로 내분하는 점을 각각 `D`, `E`, `F`라고 할 때, `triangleDEF`의 넓이를 `alpha`라 하자. `32alpha^2`의 값을 구하시오.<br/>","['4', 'ABC', 'AB', 'BC', 'CA', '1:3', 'D', 'E', 'F', 'triangleDEF', 'alpha', '32alpha^2']","['변', '길이', '정삼각형', '세', '선분', '모두', '내분하', '점', '각각', '할', '때', '넓', '하자', '값', '구하시오']",['<br/>'],삼각형의 넓이와 외접원의 반지름,NaN,7541,3,3.0,10,open,"['HSU1-07', 'HSU1-07', 'HSU1-07']","['삼각형의 넓이와 외접원의 반지름', '삼각형의 넓이', '세변의 길이를 알때 삼각형의 넓이']",HSU1-07
655,HSTA-03,HSTA-03-03,"생일이 서로 다른 `5`명이 있다. 이들을 일렬로 세울 때, 두 번째 사람이 자신과 이웃한 두 사람보다 생일이 빠를 확률은?",['5'],"['생일', '서', '다른', '명', '있', '이들', '일렬', '세울', '때', '두', '번째', '사람', '자신', '이웃', '두', '사람', '생일', '빠', '확률']",[],순열과 확률,NaN,9773,5,2.0,10,open,"['HSTA-01', 'HSTA-03', 'H1S2-07']","['중복순열', '순열과 확률', '대진표 작성하기']",HSTA-03


In [7]:
df_eval = pd.read_csv(f"{DATA_PATH}/hidden_for_inference.csv", encoding="utf-8")
df_eval.head(2)

,qplay_question_id,text
0,3535,"좌표평면 위의 두 점 `A(1, 2)`, `B(5, -2)`에 대하여 선분 `AB`의 수직이등분선 위에 한 점 `C`가 있다. `bar (AB)=bar (AC)`를 만족시킬 때, 삼각형 `ABC`의 넓이는?"
1,4537,"`a-b=3`일 때, `a^3-3a^2b+(3b^2-5)a-b^3+5b-10`의 값은?"


In [8]:
# Sample tokenizing for notations
from kobert_transformers import get_tokenizer
tokenizer = get_tokenizer()
# tokenizer.tokenize("√")
tokenizer.tokenize("|")

['▁|']

### P() -> 확률함수()로 바꾸기

In [9]:
# regex for P(a,b) type shape
text = "좌표평면 위의 원점에서 출발하는 점 `P(x, y)`는 주사위를 던져 `1`, `2`, `3`, `4`의 눈이 나오면 `x`축의 방향으로 `1`만큼, `5`, `6`의 눈이 나오면 `y`축의 방향으로 `1`만큼 움직인다. 주사위를 `20`번 던질 때, 점 `P`의 `x`좌표를 확률변수 `X`라고 하고 점 `P`의 `y`좌표를 확률변수 `Y`라고 할 때, `E(3X)+V(3Y-5)`를 구하면?"
re.findall('P\([\w\+\-\(\)/]{1,10},\s?[\w\+\-\(\)/]{1,10}\)',text)

['P(x, y)']

In [10]:
def substitute_probability(text):
    if "다항식" in text:
        return text
    if "식을 `" in text:
        return text
    if "방정식" in text:
        return text
    if "점 `" in text or "점을 `" in text:
        return text
    if (
        "점" in text
        and re.findall(r"P\([\w\+\-\(\)/]{1,10},\s?[\w\+\-\(\)/]{1,10}\)", text) != []
    ):

        return text
    else:
        substituted = re.sub("P\(", "확률함수(", text)
        return substituted

In [11]:
# copy training dataset for preprocessing
df_testing = df_train.copy()

# apply substitute probability function
df_testing["text"] = df_testing["text"].apply(lambda x: substitute_probability(x))

# remaining P() that did not go through substitute probability function
df_false_positive = df_testing[(df_testing["text"].str.contains("P\(")) & (df_testing["chapter"].str.startswith("HSTA"))]
df_false_positive

,chapter,qtid,text,math_field,hangul_field,tag_field,name,subtitle,qplay_question_id,question_difficulty,question_type_difficulty,grade,purpose,word2vec_pred_chapter,word2vec_pred_name,tfidf_pred_chapter
267,HSTA-04,HSTA-04-11,"수직선 위에 점 `P(0)`이 있다. 두 개의 동전을 던져서 모두 앞면이 나오면 점 `P`는 오른쪽으로 `1`만큼 움직이고, 그 외의 경우가 나오면 점 `P`는 왼쪽으로 `1`만큼 움직인다. 동전을 일곱 번 던졌을 때, 점 `P`의 좌표가 `3`일 확률은?","['P(0)', 'P', '1', 'P', '1', 'P', '3']","['수직선', '위', '점', '있', '두', '개', '동전', '던져', '모두', '앞면', '나오', '점', '오른쪽', '움직', '그', '외', '경우', '나오', '점', '왼쪽', '움직인', '동전', '일곱', '번', '던졌', '때', '점', '좌표', '일', '확률']",[],독립시행의 확률의 활용,NaN,9355,3,3.0,10,open,"['HSU1-07', 'H1S2-03', 'HSTA-05']","['코사인법칙의 활용', '산술평균과 기하평균의 관계 (5)', '이항분포의 평균, 분산, 표준편차 (2)']",HSTA-04
577,HSTA-05,HSTA-05-11,"좌표평면 위의 원점에서 출발하는 점 `P(x, y)`는 주사위를 던져 `1`, `2`, `3`, `4`의 눈이 나오면 `x`축의 방향으로 `1`만큼, `5`, `6`의 눈이 나오면 `y`축의 방향으로 `1`만큼 움직인다. 주사위를 `20`번 던질 때, 점 `P`의 `x`좌표를 확률변수 `X`라고 하고 점 `P`의 `y`좌표를 확률변수 `Y`라고 할 때, `E(3X)+V(3Y-5)`를 구하면?","['P(x, y)', '1', '2', '3', '4', 'x', '1', '5', '6', 'y', '1', '20', 'P', 'x', 'X', 'P', 'y', 'Y', 'E(3X)+V(3Y-5)']","['좌표평', '위', '원점', '출발하', '점', '주사위', '던져', '눈', '나오', '축', '방향', '눈', '나오', '축', '방향', '움직인', '주사위', '번', '던질', '때', '점', '좌표', '확률변수', '점', '좌표', '확률변수', '할', '때', '구하']",[],"이항분포의 평균, 분산, 표준편차 (2)",이항분포가 주어지지 않은 경우,9283,3,2.0,10,open,"['H1S1-09', 'H1S1-09', 'H1S1-09']","['선분의 내분점과 외분점', '같은 거리에 있는 점', '선분의 내분점과 외분점의 활용']",HSTA-05


In [12]:
# 잘못된 확률함수()의 대체 예시
df_true_negative = df_testing[(df_testing["text"].str.contains("확률함수\(")) & (~df_testing["chapter"].str.startswith("HSTA"))]
df_true_negative[["chapter","text"]]

,chapter,text
3224,H1S2-07,"자연수 `n`을 `k`개의 자연수로 분할하는 방법의 수를 기호 `확률함수(n, k)`로 나타낸다. <보기>에서 옳은 것만을 있는 대로 고른것은?<br/><fieldset><legend><보기></legend>ㄱ. `확률함수(7, 3)=4`<br/>ㄴ. `확률함수(10, 4)=확률함수(6, 1)+확률함수(6, 2)+확률함수(6, 3)+확률함수(6, 4)`<br/>ㄷ. `k_1 < k_2 < n`을 만족시키는 세 자연수 `k_1, k_2, n`에 대하여 `확률함수(n, k_1) <= 확률함수(n, k_2)`이다.</fieldset>"
4272,H1S2-07,"`확률함수(9, 3)`의 값은?"


In [13]:
# P()에서 몇 개나 확률함수()로 대체됐는지
df_true_positive = df_testing[(df_testing["text"].str.contains("확률함수\(")) & (df_testing["chapter"].str.startswith("HSTA"))]
len(df_true_positive)

180

### evaluation dataset에서 작동하는 지 확인

In [14]:
df_testing = df_eval.copy()
df_testing = preprocess(df_testing)
df_testing[(df_testing["text"].str.contains("확률함수 \("))].head()


,qplay_question_id,text


In [15]:
df_testing[(df_testing["text"].str.contains("P \("))]

,qplay_question_id,text


### E() -> 기댓값()으로 바꾸기

In [16]:
# reset training dataset for preprocessing
df_testing = df_train.copy()

# 일괄적으로 바꾸기
df_testing["text"] = df_testing["text"].apply(lambda x: re.sub("E\(", "기댓값(", x) if not "기울기" in x else x)  # 0개 잘못 분류

# 잘못된 E()-> 기댓값() 대체 예시
df_true_negative = df_testing[(df_testing["text"].str.contains("기댓값\(")) & (~df_testing["chapter"].str.startswith("HSTA"))]
print(len(df_true_negative))
df_true_negative

0


,chapter,qtid,text,math_field,hangul_field,tag_field,name,subtitle,qplay_question_id,question_difficulty,question_type_difficulty,grade,purpose,word2vec_pred_chapter,word2vec_pred_name,tfidf_pred_chapter


In [17]:
# 올바른 E() -> 기댓값() 대체 예시
df_true_positive = df_testing[(df_testing["text"].str.contains("기댓값\("))]
print(len(df_true_positive))
df_true_positive.head(2)

41


,chapter,qtid,text,math_field,hangul_field,tag_field,name,subtitle,qplay_question_id,question_difficulty,question_type_difficulty,grade,purpose,word2vec_pred_chapter,word2vec_pred_name,tfidf_pred_chapter
103,HSTA-05,HSTA-05-08,"한 개의 주사위를 `4`번 던져서 `3`의 배수의 눈이 나오는 횟수를 확률변수 `X`라 할 때, `기댓값(3X-1)`의 값은?","['4', '3', 'X', 'E(3X-1)']","['개', '주사위', '번', '던져', '배수', '눈', '나오', '횟수', '확률변수', '할', '때', '값']",[],"이산확률변수 aX+b의 평균, 분산, 표준편차 (3)",확률분포가 주어지지 않은 경우,9274,3,2.0,10,open,"['HSTA-05', 'HSTA-05', 'HSTA-05']","['이산확률변수 aX+b의 평균, 분산, 표준편차 (3)', '이산확률변수의 평균, 분산, 표준편차 (2)', '이항분포의 평균, 분산, 표준편차 (2)']",HSTA-05
432,HSTA-05,HSTA-05-07,"확률변수 `X`의 확률분포를 표로 나타내면 다음과 같다.<br/>`기댓값(nX+3)=10`일 때, `V(nX+1)`의 값은?","['X', 'E(nX+3)=10', 'V(nX+1)']","['확률변수', '확률분포', '표', '나타내', '다음', '같', '일', '때', '값']",['<br/>'],"이산확률변수 aX+b의 평균, 분산, 표준편차 (2)",확률분포가 주어진 경우,9302,3,2.0,10,open,"['HSTA-05', 'HSTA-05', 'HSTA-05']","['이산확률변수 aX+b의 평균, 분산, 표준편차 (2)', '이산확률변수의 평균, 분산, 표준편차 (1)', '확률질량함수의 성질 (1)']",HSTA-05


In [18]:
# 아직 안 바뀐 E() 들
df_true_negatives = df_testing[(df_testing["text"].str.contains("E\("))]
print(len(df_true_negatives))
df_true_negatives.head(2)

1


,chapter,qtid,text,math_field,hangul_field,tag_field,name,subtitle,qplay_question_id,question_difficulty,question_type_difficulty,grade,purpose,word2vec_pred_chapter,word2vec_pred_name,tfidf_pred_chapter
1792,H1S1-09,H1S1-09-06,"다음은 임의의 네 점 `A(x_1,y_1)`, `B(x_2,y_2)`, `C(x_3,y_3)`, `D(x_4,y_4)`를 꼭짓점으로 하는 사각형에서 각 변의 중점을 꼭짓점으로 하는 사각형 `EFGH`가 어떤 사각형이 되는지 확인하는 과정이다. 빈칸을 채운 것으로 알맞지 않은 것은?<fieldset><legend></legend>네 변 `bar (AB)`, `bar (BC)`, `bar (CD)`, `bar (DA)`의 중점을 각각 `E`, `F`, `G`, `H`라 두면 네 점 `E`, `F`, `G`, `H`의 좌표는 각각<br/>`E(①)`, `F((x_2+x_3)/2,(y_2+y_3)/2)`, `G((x_3+x_4)/2,(y_3+y_4)/2)`. `H((x_4+x_1)/2,(y_4+y_1)/2)`이다. 이때, `bar (EF)`의 기울기는 ②, `bar (FG)`의 기울기는 ③, `bar (GH)`의 기울기는 ②, `bar (HE)`의 기울기는 ③이다. 따라서 `bar (EF)④bar (GH)`, `bar (FG)④bar (HE)`이므로 사각형 `EFGH`는 ⑤이다.</fieldset>","['A(x_1,y_1)', 'B(x_2,y_2)', 'C(x_3,y_3)', 'D(x_4,y_4)', 'EFGH', 'bar (AB)', 'bar (BC)', 'bar (CD)', 'bar (DA)', 'E', 'F', 'G', 'H', 'E', 'F', 'G', 'H', 'E(①)', 'F((x_2+x_3)/2,(y_2+y_3)/2)', 'G((x_3+x_4)/2,(y_3+y_4)/2)', 'H((x_4+x_1)/2,(y_4+y_1)/2)', 'bar (EF)', 'bar (FG)', 'bar (GH)', 'bar (HE)', 'bar (EF)④bar (GH)', 'bar (FG)④bar (HE)', 'EFGH']","['다음', '임의', '점', '꼭짓점', '하', '사각형', '각', '변', '중점', '꼭짓점', '하', '사각형', '어떤', '사각형', '되는', '확인하', '과정', '빈칸', '채운', '것', '알맞', '않', '것', '변', '중점', '각각', '두', '점', '좌표', '각각', '이때', '기울기', '기울기', '기울기', '기울기', '따라', '이므', '사각형']","['<fieldset>', '<legend>', '</legend>', '<br/>', '</fieldset>']",좌표를 이용한 도형의 성질 판단,NaN,4428,3,2.0,10,open,"['H1S1-09', 'H1S1-09', 'HSU1-06']","['좌표를 이용한 도형의 성질 판단', '거리의 제곱의 합의 최솟값', '일반각에서 삼각함수 성질 활용하기']",H1S1-09


## Evaluation Dataset에서도 올바르게 작동하는지 확인

In [19]:
df_testing = df_eval.copy()

df_E = df_testing[(df_testing["text"].str.contains("E\("))]
print(len(df_E))
df_E

5


,qplay_question_id,text
165,9261,"한 개의 주사위를 `40`번 던질 때, 소수의 눈이 나오는 횟수를 확률변수 `X`라 하자. `E(3X-1)`의 값은?"
169,9300,"다음 그림과 같이 한 모서리의 길이가 `1`인 정육면체에서 세 꼭짓점을 택하여 삼각형을 만들려고 한다. 만들어지는 삼각형의 넓이의 `2`배를 확률변수 `X`라고 할 때, `E(7X-sqrt3)`을 구하면?"
951,9168,"어느 모집단의 확률변수 `X`의 확률질량함수가 `P(X=r)={::}_90C_r(1/3)^r(2/3)^(90-r)` `(r=0, 1, 2, cdots, 90)`이고, 이 모집단에서 크기가 `5`인 표본을 임의추출하여 구한 표본평균을 `bar(X)`라 할 때, `E(bar(X))+V(bar(X))`의 값은?"
963,9368,"확률변수 `X`의 확률분포를 표로 나타내면 다음과 같다.<br/>`(E(X))/a`의 값은? (단, `a`는 상수이다.)"
1259,9288,"확률변수 `X`의 확률질량함수가 `P(X=x)={::}_100C_x7^x/2^300 (x=0, 1, 2, cdots, 100)`일 때, `E(8X+100)`의 값을 구하시오."


In [20]:
df_testing = df_eval.copy()
df_testing["text"] = df_testing["text"].apply(lambda x: re.sub("E\(", "기댓값(", x) if not "기울기" in x else x)  # 0개 잘못 분류

df_expectation = df_testing[(df_testing["text"].str.contains("기댓값\("))]
print(len(df_expectation))
df_expectation

5


,qplay_question_id,text
165,9261,"한 개의 주사위를 `40`번 던질 때, 소수의 눈이 나오는 횟수를 확률변수 `X`라 하자. `기댓값(3X-1)`의 값은?"
169,9300,"다음 그림과 같이 한 모서리의 길이가 `1`인 정육면체에서 세 꼭짓점을 택하여 삼각형을 만들려고 한다. 만들어지는 삼각형의 넓이의 `2`배를 확률변수 `X`라고 할 때, `기댓값(7X-sqrt3)`을 구하면?"
951,9168,"어느 모집단의 확률변수 `X`의 확률질량함수가 `P(X=r)={::}_90C_r(1/3)^r(2/3)^(90-r)` `(r=0, 1, 2, cdots, 90)`이고, 이 모집단에서 크기가 `5`인 표본을 임의추출하여 구한 표본평균을 `bar(X)`라 할 때, `기댓값(bar(X))+V(bar(X))`의 값은?"
963,9368,"확률변수 `X`의 확률분포를 표로 나타내면 다음과 같다.<br/>`(기댓값(X))/a`의 값은? (단, `a`는 상수이다.)"
1259,9288,"확률변수 `X`의 확률질량함수가 `P(X=x)={::}_100C_x7^x/2^300 (x=0, 1, 2, cdots, 100)`일 때, `기댓값(8X+100)`의 값을 구하시오."


### 집합의 조건제시법 |, 조건부확률의 |, 절대값 || 다르게 처리

In [21]:
def preprocess_pipe(input_text: str) -> dict:
    list_fucked_up = []
    list_jiphap = []
    list_conditional = []
    list_absolute = []

    list_math_matched = re.findall(r"(?<=`).*?(?=`)", input_text)
    list_math = [
        math for math in list_math_matched if list_math_matched.index(math) % 2 == 0
    ]
    for item in list_math:
        if "|" in item and "{" in item:
            # print(item, "집합")
            list_jiphap.append(item)
        elif "|" in item and "(" in item and ")" in item:
            # print("conditional")
            list_conditional.append(item)
        elif item.count("|") >= 2:
            list_absolute.append(item)
            # print(item, "absolute_value")
    if len(list_jiphap)*len(list_conditional)*len(list_absolute) !=0:
        return "FUCKED"
    if len(list_jiphap+list_conditional+list_absolute) ==0:
        return np.NaN
    else: 
        dict_return = {"집합": list_jiphap, "조건부":list_conditional, "절댓값": list_absolute}
        return dict_return


In [22]:
sample_text3 = "a, b, n, `|a+b|=|a|+|b|`, ab >= 0, a^2+b^2=0, ab=0, ab=0, a=0, b=0, n^2, 3, n, 3"                                                       
preprocess_pipe(sample_text3)

{'집합': [], '조건부': [], '절댓값': ['|a+b|=|a|+|b|']}

In [23]:
re.sub("\|","테스트|",sample_text3)

'a, b, n, `테스트|a+b테스트|=테스트|a테스트|+테스트|b테스트|`, ab >= 0, a^2+b^2=0, ab=0, ab=0, a=0, b=0, n^2, 3, n, 3'

In [24]:
df_train["pipes"] = df_train["text"].apply(lambda x: preprocess_pipe(x))

In [25]:
# 집합, 조건부, 절댓값이 겹치는 경우가 없음.
# df_fucked = df_train[df_train["pipes"].str.match("FUCKED")]

In [26]:
pd.set_option('display.max_rows', 20)
df_pipe = df_train[df_train["pipes"].notna()]
# df_pipe[0:300]
df_pipe = df_pipe.set_index("qplay_question_id")
df_pipe.sample(20)

,chapter,qtid,text,math_field,hangul_field,tag_field,name,subtitle,question_difficulty,question_type_difficulty,grade,purpose,word2vec_pred_chapter,word2vec_pred_name,tfidf_pred_chapter,pipes
qplay_question_id,,,,,,,,,,,,,,,,
24332,H1S2-06,H1S2-06-10,함수 `y = sqrt(ax) (a != 0)`에 대한 설명 중 옳은 것은?<br/><fieldset>ㄱ) 정의역은 `{x | x >= 0}`이다.<br/>ㄴ) 치역은 `{y | y >= 0}`이다.<br/>ㄷ) 그래프는 제`1`사분면을 지난다.<br/>ㄹ) 함수 `y = -sqrt(ax) (a!= 0)`의 그래프와 `x`축에 대하여 대칭이다.</fieldset>,"['y = sqrt(ax) (a != 0)', '{x | x >= 0}', '{y | y >= 0}', '1', 'y = -sqrt(ax) (a!= 0)', 'x']","['함수', '대', '설명', '중', '옳', '것', '정의역', '치역', '그래프', '제', '사분면', '지난', '함수', '그래프', '축', '대하', '대칭']","['<br/>', '<fieldset>', '<br/>', '<br/>', '<br/>', '</fieldset>']",무리함수의 그래프의 성질,NaN,2,NaN,10,open,"['HSU1-03', 'H1S2-06', 'H1S2-05']","['지수함수의 성질과 그래프', '무리함수의 그래프의 성질', '유리함수의 그래프의 성질']",H1S2-06,"{'집합': ['{x | x >= 0}', '{y | y >= 0}'], '조건부': [], '절댓값': []}"
25160,H1S2-04,H1S2-04-15,"집합 `X={x|0 <= x <= 2}`에 대하여 `X`에서 `X`로의 함수 `y=f(x)`의 그래프가 아래 그림과 같을 때, 방정식 `(f@f@f)(x)=f(x)`의 실근의 개수는?","['X={x|0 <= x <= 2}', 'X', 'X', 'y=f(x)', '(f@f@f)(x)=f(x)']","['집합', '대하', '로', '함수', '그래프', '아', '그림', '같', '때', '방정식', '실근', '개수']",[],함수의 그래프와 합성함수,NaN,4,NaN,10,open,"['H1S2-04', 'H1S2-04', 'H1S2-04']","['함수의 그래프와 합성함수', 'f∘g=g∘f인 경우', '역함수가 존재하기 위한 조건']",H1S2-04,"{'집합': ['X={x|0 <= x <= 2}'], '조건부': [], '절댓값': []}"
25301,H1S2-04,H1S2-04-06,"집합 `X={x|0<=x<=3}`에서 `X`로의 함수<br/>`f(x)={(ax, (0<=x<1)),(bx+c, (1<=x<=3)):}`이 일대일 대응일 때, <보기>에서 옳은 것만을 모두 고르면? (단, `a`, `b`, `c`는 상수)<fieldset><legend><보기></legend>ㄱ. `0 < a <3`<br/>ㄴ. `ab >0`이면 `a+2b=3`<br/>ㄷ. `ab < 0`이면 `a+2c=9`</fieldset>","['X={x|0<=x<=3}', 'X', 'f(x)={(ax, (0<=x<1)),(bx+c, (1<=x<=3)):}', 'a', 'b', 'c', '0 < a <3', 'ab >0', 'a+2b=3', 'ab < 0', 'a+2c=9']","['집합', '로', '함수', '일대일', '대응일', '때', '보기', '옳', '것', '모두', '고르', '단', '상수', '보기', '이', '이']","['<br/>', '<보기>', '<fieldset>', '<legend>', '<보기>', '</legend>', '<br/>', '<br/>', '</fieldset>']",일대일대응 찾기,NaN,4,NaN,10,open,"['H1S2-04', 'H1S2-04', 'H1S2-04']","['함수의 정의역, 공역, 치역', '역함수가 존재하기 위한 조건', '일대일대응이 되도록 하는 미지수 구하기']",H1S2-04,"{'집합': ['X={x|0<=x<=3}'], '조건부': [], '절댓값': []}"
9138,HSTA-07,HSTA-07-07,"어느 공장에서 생산되는 제품 한 개의 무게는 평균 `m``g`, 표준편차 `10g`인 정규분포를 따른다고 한다. 이 공장에서 생산된 제품 중 `400`개를 임의추출하여 그 무게를 측정하였더니 평균이 `80g`이었다. 이 공장에서 생산되는 제품의 무게의 모평균 `m`에 대하여 신뢰도 `95%`인 신뢰구간을 구하여라. (단, `Z`가 표준정규분포를 따르는 확률변수일 때, `P(|Z| <= 1.96)=0.95`로 계산한다.)","['m', '`g', '10g', '400', '80g', 'm', '95%', 'Z', 'P(|Z| <= 1.96)=0.95']","['어느', '공장', '생산되', '제품', '개', '무게', '평균', '표준편차', '정규분포', '따른다', '한', '공장', '생산된', '제품', '중', '개', '임의추출하', '그', '무게', '측정하였더', '평균', '이었', '공장', '생산되', '제품', '무게', '모평균', '대하', '신뢰', '신뢰구간', '구하여', '단', '표준정규분포', '따르', '확률변수일', '때', '계산한']",[],모평균의 추정 (1),모표준편차가 주어진 경우,3,3.0,10,open,"['HSTA-07', 'HSTA-07', 'HSTA-07']","['모평균의 추정 (2)', '신뢰구간의 길이 (1)', '모평균의 추정 (3)']",HSTA-07,"{'집합': [], '조건부': ['P(|Z| <= 1.96)=0.95'], '절댓값': []}"
9433,HSTA-04,HSTA-04-01,"두 사건 `A`, `B`에 대하여 `P(A^C) = 3/5`, `P(A^C nn B^C) = 1/5`, `P(B|A) = 1/2`일 때, `P(A|B)`의 값은?","['A', 'B', 'P(A^C) = 3/5', 'P(A^C nn B^C) = 1/5', 'P(B|A) = 1/2', 'P(A|B)']","['두', '사건', '대하', '일', '때', '값']",[],조건부확률의 계산,NaN,3,2.0,10,open,"['HSTA-04', 'HSTA-04', 'HSTA-03']","['조건부확률의 계산', '독립사건의 확률', '확률의 덧셈정리와 여사건의 확률']",HSTA-04,"{'집합': [], '조건부': ['P(B|A) = 1/2', 'P(A|B)'], '절댓값': []}"
3496,H1S1-08,H1S1-08-09,`x`에 대한 부등식 `|8x-5|+7 > a`의 해가 모든 실수가 되도록 하는 자연수 `a`의 최댓값은?,"['x', '|8x-5|+7 > a', 'a']","['대', '부등식', '해', '모든', '실수', '되도록', '하', '자연수', '최댓값']",[],특수한 해를 갖는 일차부등식,NaN,3,2.0,10,open,"['H1S1-08', 'H1S1-08', 'H1S1-08']","['특수한 해를 갖는 일차부등식', '절댓값 또는 가우스 기호가 포함된 이차부등식', '절댓값 기호가 포함된 일차부등식']",H1S1-08,"{'집합': [], '조건부': [], '절댓값': ['|8x-5|+7 > a']}"
25085,H1S2-04,H1S2-04-15,"집합 `X = {x|0 <= x <= 2}`에서 정의된 두 함수 `y = f(x)`와 `y = g(x)`의 그래프가 그림과 같을 때, `0 <= x <= 2`에서 방정식 `f(g(x)) = x`를 만족하는 서로 다른 실근의 개수는?","['X = {x|0 <= x <= 2}', 'y = f(x)', 'y = g(x)', '0 <= x <= 2', 'f(g(x)) = x']","['집합', '정의된', '두', '함수', '그래프', '그림', '같', '때', 

In [27]:
# def sub_func_replace_pipe(input_text: str) -> str:
#     if "|" in input_text and "{" in input_text:
#         input_text = re.sub("\|", "조건제시법|", input_text)
#     elif "|" in input_text and "확률함수(" in input_text and ")" in input_text:
#         if input_text.count("|") % 2 == 0:
#             input_text = re.sub("\|", "절대값|", input_text)
#         if input_text.count("|") % 2 == 1:
#             input_text = re.sub("\|", "조건부확률|", input_text)
#     elif input_text.count("|") >= 2:
#         input_text = re.sub("\|", "절대값|", input_text)
#     else:
#         return input_text
#     result_text = input_text
#     return result_text

def sub_func_replace_pipe(input_text: str) -> str:
    if "|" in input_text and "{" in input_text:
        input_text = re.sub("\|", "조건제시법|", input_text)
    elif "|" in input_text and "확률함수(" in input_text and ")" in input_text:
        words_broken_down = []
        # word_broken_by_bracket = re.findall(r'\([^)]*\)', input_text)
        word_broken_by_bracket = re.findall('\[[^\]]*\]|\([^\)]*\)|\"[^\"]*\"|\S+',input_text)
        print(word_broken_by_bracket)
        for i in word_broken_by_bracket:
            print(i)
            if i.count("|") % 2 == 0:
                i = re.sub("\|", "절대값|", i)
                words_broken_down.append(i)
            elif i.count("|") % 2 == 1:
                i = re.sub("\|", "조건부확률|", i)
                words_broken_down.append(i)
            else:
                words_broken_down.append(i)
        input_text = "".join(words_broken_down)
    elif input_text.count("|") >= 2:
        input_text = re.sub("\|", "절대값|", input_text)
    else:
        return input_text
    result_text = input_text
    return result_text

In [28]:

#  if list_math_matched.index(math) % 2 == 0
def main_replace_pipe(input_text: str) -> str:
    # print(input_text, "\n")

    list_to_return = []
    # list_separated = re.findall(r'(?<=`).*?(?=`)', input_text)
    # list_separated = input_text.split("`")

    delimiter = "`"
    if input_text.startswith(delimiter) == True:
        bool_starts_with_delimiter = True
    elif input_text.startswith(delimiter) == False:
        bool_starts_with_delimiter = False
    list_separated = [delimiter + e for e in input_text.split(delimiter) if e]

    # print(list_separated, "\n")
    for item in list_separated:
        if "|" in item and "{" in item:
            item = sub_func_replace_pipe(item)
            # item = f"`{item}`"
            list_to_return.append(item)
        elif "|" in item and "확률함수(" in item and ")" in item:
            item = sub_func_replace_pipe(item)
            # item = f"`{item}`"
            list_to_return.append(item)
        elif item.count("|") >= 2:
            item = sub_func_replace_pipe(item)
            # item = f"`{item}`"
            list_to_return.append(item)
        else:
            list_to_return.append(item)
    # print(list_to_return, "\n")
    str_to_return = "".join(list_to_return)
    if bool_starts_with_delimiter == True:
        return str_to_return
    elif bool_starts_with_delimiter == False:
        return str_to_return[1:]

In [29]:
main_replace_pipe("정규분포 `N(m, 9)`에 따르는 확률변수 `X`에 대하여 `확률함수(|X| <= 63)=0.6826`일 때, 상수 `m`의 값은?")

['`확률함수(|X|', '<=', '63)=0.6826']
`확률함수(|X|
<=
63)=0.6826


'정규분포 `N(m, 9)`에 따르는 확률변수 `X`에 대하여 `확률함수(절대값|X절대값|<=63)=0.6826`일 때, 상수 `m`의 값은?'

In [30]:
import random

rand_int = random.randrange(0, 101)
sample_text = df_pipe["text"].iloc[rand_int]
print(main_replace_pipe(sample_text))

이차함수 `y=3x^2+2`의 그래프와 직선 `y=3x+절대값|a-1절대값|`이 만나지 않도록 하는 실수 `a`의 값의 범위는 `m< a< M`과 같다. `M/m`의 값을 구하시오.


In [31]:
df_test_pipe = preprocess(df_original, korean=True, space=True, condition=True)
df_test_pipe = df_test_pipe.set_index("qplay_question_id")
df_test_pipe.sample(2)

,text,qtid,name,subtitle,question_difficulty,question_type_difficulty,grade,purpose
qplay_question_id,,,,,,,,
6993,"삼각형 `ABC`의 세 내각의 크기를 각각 `A`, `B`, `C`라 할 때, ` tan A=-1/2`이다. 이때 `√5 cos (B+C)`의 값을 구하시오.",HSU1-06-12,여러 가지 각의 삼각함수 값 (3),도형에서 활용하기,2,3.0,10,open
7948,"` theta `가 제`1`사분면의 각일 때, 각 ` theta / 2`가 속하는 사분면으로 옳은 것은?",HSU1-05-02,사분면의 각,NaN,3,1.0,10,open


In [44]:
df_test_pipe_lookup = df_test_pipe.loc[df_pipe.index]
print(df_test_pipe_lookup.shape)
df_test_pipe_lookup.sample(15)

(323, 8)


,text,qtid,name,subtitle,question_difficulty,question_type_difficulty,grade,purpose
qplay_question_id,,,,,,,,
24882,"집합 `X = {x 조건제시법| 0 ≤ x ≤ 1}`에 대하여 `X`에서 `X`로의 함수 `y = f(x)`의 그래프가 그림과 같다. `f^2 = f@f`, `f^3 = f^2 @f`, `f^4 = f^3 @ f …` 이라 할 때, `f(1/4) + f^2(1/4) + f^3(1/4) + … + f^40(1/4)`의 값은? `, ` bar (AB)`는 선분이다.)",H1S2-04-14,fⁿ 꼴의 합성함수,NaN,4,NaN,10,open
18846,"다음 보기 중 무리함수 `y=√(-2x+6)+1`의 그래프에 대한 설명으로 옳은 것을 모두 고른 것은? 보기 정의역은 `{x조건제시법|x≤3}`, 치역은 `{y조건제시법|y≥1}`이다. `y=√(-2x)`의 그래프를 `x`축의 방향으로 `6`만큼, `y`의 방향으로 `1`만큼 평행이동한 그래프이다. 제`3`, `4`사분면을 지나지 않는다.",H1S2-06-10,무리함수의 그래프의 성질,NaN,3,NaN,10,open
24262,"집합 `X={x조건제시법|0 ≤ x ≤2}`일 때 `X`에서 `X`로의 함수 `f(x)=ax^2+bx`의 역함수가 존재하도록 하는 실수 `a`, `b`에 대하여 좌표평면에서 점 `(a, b)`가 나타내는 도형의 길이를 구하시오.",H1S2-04-17,역함수가 존재하기 위한 조건,NaN,4,NaN,10,open
17809,"전체집합 `U={x조건제시법|x`는 `10`이하의 자연수`}`의 두 부분집합 `A={x조건제시법|x`는 `8`의 약수`}`, `B={x조건제시법|x`는 `10`이하의 짝수`}`에 대하여 `A여집합-B`의 원소의 개수는?",H1S2-02-02,여집합과 차집합,NaN,2,NaN,10,open
9430,"두 사건 `A`, `B`에 대하여 `확률함수(A) = 0.6`, `확률함수(B) = 0.2`, `확률함수(A ∪ B) = 0.7`일 때, `확률함수(B조건부확률|A여집합)`는?",HSTA-04-01,조건부확률의 계산,NaN,3,2.0,10,open
24755,"함수 `f(x)=mx-1`의 정의역이 `{x조건제시법|2 ≤ x ≤ 3}`이고 공역이 `{y조건제시법| 1 ≤ y ≤ 5}`일 때, 실수 `m`의 값의 범위는 `a ≤ m ≤ b`이다. 이때, 상수 `a`, `b`에 대하여 `a+b`의 값은?",H1S2-04-03,"함수의 정의역, 공역, 치역",NaN,3,NaN,10,open
7405,"함수 `y=절대값| cos x/2절대값|-1`의 주기와 최댓값, 최솟값을 옳게 구한 것을 고르시오.",HSU1-06-17,"삼각함수를 포함한 식의 최대, 최소 (4)",절댓값 기호를 포함한 경우,3,3.0,10,open
25245,"두 집합 `X={-1, 0, 1, 2}`, `Y={0, 1, 2, 3, 4}`에 대하여 `X`에서 `Y`로의 함수인 것을 보에서 있는 대로 고른 것은? 보 `y=x` `y=절대값|x절대값|+1` `y=x^2` `y=x^2-x+1`",H1S2-04-01,함수의 뜻,NaN,3,NaN,10,open
3487,부등식 `절대값|x절대값|^2-4절대값|x절대값|-12 ≤ 0`을 만족시키는 정수 `x`의 개수를 구하시오.,H1S1-08-11,절댓값 또는 가우스 기호가 포함된 이차부등식,NaN,3,2.0,10,open


In [49]:
df_test_pipe_lookup[df_test_pipe_lookup["qtid"].str.startswith("HSTA")].sample(20)

,text,qtid,name,subtitle,question_difficulty,question_type_difficulty,grade,purpose
qplay_question_id,,,,,,,,
9143,어느 고등학교 `2`학년 학생들의 키는 표준편차가 `10cm`인 정규분포를 따른다고 한다. 이 고등학교 `2`학년 학생 중 `64`명을 임의추출하여 키를 조사하였더니 평균이 `160cm`이었다. 이 고등학교 `2`학년 전체 학생의 키의 평균 `m`에 대한 신뢰도 `95%`의 신뢰구간을 `l`이라 하자. `10l`의 값을 구하여라. =0.95`),HSTA-07-07,모평균의 추정 (1),모표준편차가 주어진 경우,3,3.0,10,open
9744,"어느 공장에서 만드는 제품 한 개의 무게는 표준편차가 `5g`인 정규분포를 따른다고 한다. 이 공장에서 만든 제품 중에서 `n`개를 임의추출하여 그 무게를 조사하였더니 평균이 `30g`이었다. 이 공장에서 만드는 제품의 평균 무게 `mg`을 신뢰도 `99%`로 추정한 신뢰구간이 `27.85 ≤ m ≤ 32.15`일 때, `n`의 값은? =0.99`)",HSTA-07-09,모평균의 추정 (3),표본의 크기 계산하기,3,3.0,10,open
9115,"표준편차가 `sigma`인 정규분포를 따르는 어느 모집단에서 크기가 `n` 인 표본을 임의추출한 신뢰도 `95%`의 신뢰구간의 길이가 `l`일 때, 크기가 `16n`인 표본을 임의추출하여 추정한 신뢰도 `95%`의 신뢰구간의 길이는? =0.95`로 계산한다.)",HSTA-07-10,신뢰구간의 길이 (1),NaN,3,2.0,10,open
9150,"정규분포을 따르는 모집단에서 크기가 `n`인 표본을 임의추출할 때, 그 표본평균을 ` bar X`라고 하자. `확률함수(29 ≤ bar X ≤ 31)=0.95`를 만족하는 `n`의 값은? =0.95`)",HSTA-07-05,표본평균의 확률 (2),표본의 크기 계산하기,3,2.0,10,open
9727,"표본공간 `S={x조건제시법|x`는 10 이하의 자연수`}`에 대하여, 두 사건 `A`, `B`가 `A={x조건제시법|x`는 `절대값|x절대값| ≤ 6`인 자연수`}`, `B={x조건제시법|x`는 소수`}`일 때, 표본공간 `S`의 사건 중에서 `A`, `B`와 모두 배반사건인 것은 `n`개이다. 자연수 `n`의 값을 구하시오.",HSTA-03-01,시행과 사건,NaN,2,1.0,10,open
8008,"어느 고등 학교 학생의 `1`인당 연간 독서량은 표준편차가 `6`권인 정규분포를 따른다. 학생 중 임의로 `n`명을 뽑아 조사한 `1`인당 연간 독서량의 평균이 `10`권일 때, 전체 학생의 `1`인당 연간 독서량의 평균 `m`을 신뢰도 `95%`로 추정한 신뢰구간이 `8 ≤ m ≤ 12`이었다. 이 때, 표본의 크기 `n`의 값을 구하여라. =0.95`)",HSTA-07-07,모평균의 추정 (1),모표준편차가 주어진 경우,3,3.0,10,open
9537,집합 `A={x조건제시법|x`는 `12` 이하의 자연수`}`의 부분집합 중 원소의 개수가 홀수인 것의 개수는?,HSTA-02-09,이항계수의 성질,NaN,3,3.0,10,open
9686,"어느 과수원에서 생산하는 사과 한 개의 무게는 모평균이 `m`, 모표준편차가 `10`인 정규분포를 따른다고 한다. 이 과수원에서 생산하는 사과의 무게를 조사하기 위하여 임의로 `64`개를 택하여 조사하였다. 이 과수원에서 생산하는 사과 한 개의 무게의 모평균 `m`에 대한 신뢰도 `95%`의 신뢰구간이 `a ≤ m ≤b`일 때, `b-a`의 값은? =0.95`로 계산한다.)",HSTA-07-10,신뢰구간의 길이 (1),NaN,3,2.0,10,open
7791,"어느 고등 학교 학생 전체의 키의 표준편차는 `5cm`라고 한다. 이 학교의 학생 중 `100`명을 임의추출하여 키를 조사하였더니 평균 `167cm`라고 할 때, 이 고등 학교 학생 전체의 키의 평균을 `95%`의 신뢰도로 추정하면? =0.95`)",HSTA-07-07,모평균의 추정 (1),모표준편차가 주어진 경우,3,3.0,10,open


In [47]:
df_train.loc[df_train["qplay_question_id"] == 9727][["text"]]

,text
1979,"표본공간 `S={x|x`는 10 이하의 자연수`}`에 대하여, 두 사건 `A`, `B`가 `A={x|x`는 `|x| <= 6`인 자연수`}`, `B={x|x`는 소수`}`일 때, 표본공간 `S`의 사건 중에서 `A`, `B`와 모두 배반사건인 것은 `n`개이다. 자연수 `n`의 값을 구하시오."


In [34]:
df_train.loc[df_train["qplay_question_id"] == 9318][["text"]]

,text
2739,"정규분포 `N(m, 9)`에 따르는 확률변수 `X`에 대하여 `P(|X| <= 63)=0.6826`일 때, 상수 `m`의 값은?"


In [35]:
df_test_pipe_lookup.loc[9318][["text"]]

text    정규분포에 따르는 확률변수 `X`에 대하여 `확률함수(절대값|X절대값|≤63)=0.6826`일 때, 상수 `m`의 값은?
Name: 9318, dtype: object

In [36]:
df_train.loc[df_train["qplay_question_id"] == 9146][["text"]]

,text
548,"정규분포 `N(m ,4^2)`을 따르는 모집단에서 임의추출한 크기가 `64`인 표본의 표본평균이 `32`일 때, 모평균 `m`의 신뢰도 `95%`의 신뢰구간은? (단, `P(|Z| <= 2)=0.95`)"


In [37]:
df_test_pipe.loc[9146][["text"]]

text    정규분포을 따르는 모집단에서 임의추출한 크기가 `64`인 표본의 표본평균이 `32`일 때, 모평균 `m`의 신뢰도 `95%`의 신뢰구간은? =0.95`)
Name: 9146, dtype: object

In [38]:
problem_text = "정규분포 `N(m ,4^2)`을 따르는 모집단에서 임의추출한 크기가 `64`인 표본의 표본평균이 `32`일 때, 모평균 `m`의 신뢰도 `95%`의 신뢰구간은? (단, `확률함수(|Z| <= 2)=0.95`)"
main_replace_pipe(problem_text)

['`확률함수(|Z|', '<=', '2)=0.95']
`확률함수(|Z|
<=
2)=0.95


'정규분포 `N(m ,4^2)`을 따르는 모집단에서 임의추출한 크기가 `64`인 표본의 표본평균이 `32`일 때, 모평균 `m`의 신뢰도 `95%`의 신뢰구간은? (단, `확률함수(절대값|Z절대값|<=2)=0.95`)'

In [39]:
df_eval_test_pipe = preprocess(df_eval, korean=True, space=True, condition=True)